In [ ]:
import numpy as np
from scipy.optimize import linprog
from scipy import sparse

# === Your grid setup code stays the same up to constraint building ===
# But when you build each A_*, use sparse.lil_matrix instead of dense np.zeros

num_vars = grid_size ** 3

# SPX T1 calls
A_S1 = sparse.lil_matrix((len(S1_strikes), num_vars))
for k, K1 in enumerate(S1_strikes):
    for i in range(grid_size):
        for j in range(grid_size):
            for l in range(grid_size):
                idx = i * grid_size * grid_size + j * grid_size + l
                payoff = s1_vals[i] - K1
                if payoff > 0:
                    A_S1[k, idx] = payoff
A_S1 = A_S1.tocsr()

# SPX T2 calls
A_S2 = sparse.lil_matrix((len(S2_strikes), num_vars))
for k, K2 in enumerate(S2_strikes):
    for i in range(grid_size):
        for j in range(grid_size):
            for l in range(grid_size):
                idx = i * grid_size * grid_size + j * grid_size + l
                payoff = s2_vals[j] - K2
                if payoff > 0:
                    A_S2[k, idx] = payoff
A_S2 = A_S2.tocsr()

# Martingale constraint
rows, cols, data = [], [], []
for i in range(grid_size):
    for l in range(grid_size):
        s1 = s1_vals[i]
        for j in range(grid_size):
            idx = i * grid_size * grid_size + j * grid_size + l
            rows.append(i * grid_size + l)
            cols.append(idx)
            data.append(s2_vals[j] - s1)
A_mart = sparse.csr_matrix((data, (rows, cols)), shape=(grid_size*grid_size, num_vars))
b_mart = np.zeros(grid_size * grid_size)

# Dispersion constraint
rows, cols, data = [], [], []
for i in range(grid_size):
    for l in range(grid_size):
        s1 = s1_vals[i]
        v = v_vals[l]
        for j in range(grid_size):
            idx = i * grid_size * grid_size + j * grid_size + l
            r = max(s2_vals[j] / s1, 1e-8)
            rows.append(i * grid_size + l)
            cols.append(idx)
            data.append(L(r) - v**2)
A_disp = sparse.csr_matrix((data, (rows, cols)), shape=(grid_size*grid_size, num_vars))
b_disp = np.zeros(grid_size * grid_size)

# Normalisation
A_norm = sparse.csr_matrix(np.ones((1, num_vars)))
b_norm = np.array([1.0])

# --- Stack constraints ---
A_eq = sparse.vstack([A_S1, A_S2, A_norm, A_mart, A_disp]).tocsr()
b_eq = np.concatenate([C1, C2, b_norm, b_mart, b_disp])
bounds = [(0, None)] * num_vars

# --- Example payoff ---
payoff_grid = np.maximum(V - V_forw, 0)
payoff_flat = payoff_grid.flatten()

# --- Solve LP ---
res_min = linprog(
    c=payoff_flat,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="highs"
)
print("Min price:", res_min.fun if res_min.success else np.nan)

res_max = linprog(
    c=-payoff_flat,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method="highs"
)
print("Max price:", -res_max.fun if res_max.success else np.nan)
